In [1]:
#pip install chart-studio
#create a plotly account     #https://plotly.com/python/getting-started-with-chart-studio/
from chart_studio.tools import set_config_file
set_config_file(plotly_domain="https://plotly.com", plotly_api_domain="https://api.plotly.com")


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"
import plotly
import numpy as np
import chart_studio


from Pneumonia_models.DAF_CapsNet import base_model
#from models_sabbour import base_model
from train2 import margin_loss
import matplotlib.pyplot as plt
from hyperparams import *
#plotly.tools.set_credentials_file(username= 'pmensah' , api_key= 'FrC8AAZ66YA6Wdk8v5U1')
#chart_studio.tools.set_credentials_file(username='pmensah', api_key= 'FrC8AAZ66YA6Wdk8v5U1')
#Y21nXq2ib46WqQGGYUM9
chart_studio.tools.set_credentials_file(username='username', api_key= 'api_key') ## Insert your "surname and api-key"
plt.rcParams['axes.linewidth']=1.5
plt.rcParams.update({'font.size':14})

In [ ]:
from utils import load_mnist, load_other,load_cifar10, load_fashion_mnist, load_other
from hyperparams import * 

#from Pneumonia_models.BSCapsModel import base_model
#from Pneumonia_models.DSG_Caps import base_model
#from Pneumonia_models.Squeeze_Caps import base_model

#from Pneumonia_models.CapsNet import base_model

from Pneumonia_models.DAF_CapsNet import base_model
#from Pneumonia_models.BSCapsModel import base_model
#from models_sabbour import base_model
from train2 import margin_loss
# parameters
routing = ROUTING
model_path = MODEL_PATH
#model.path='E:/DeepLearning/envs/myenvs/kobby/SCCapsule/models/Brain Tumor Results/DAF+CLAHE/Segmented Data/No Augmentation/DAF_CapsNet_model'

# load data
#(x_train, y_train), (x_test, y_test) = load_cifar10()
#(x_train, y_train), (x_test, y_test) = load_fashion_mnist()
(x_train, y_train), (x_test, y_test) = load_other()

x_shape = x_train[0].shape
y_shape = y_train[0].shape


#x_shape=[x_shape[0],x_shape[1], x_shape[2]]

print(x_shape)
# define model
train_model, decoder, digitcaps_model, pc_model = base_model(input_shape = x_shape, output_shape = y_shape)#in order of creation in models.py
#train_model , eval_model, caps_model, decoder, primCap_model = base_model(input_shape = x_shape, output_shape = y_shape)
#train_model.load_weights('E:/DeepLearning/envs/myenvs/kobby/SCCapsule/MLAF_No_CLAHE/DAF_CapsNet_model_weights.best')
#train_model.load_weights('E:/DeepLearning/envs/myenvs/kobby/SCCapsule/models/weights/DSG_CapsNet_model_weights.best')
#train_model.load_weights('DSG_CapsNet_model_weights.best.hdf5')


In [ ]:
## Import Model weights

In [ ]:
train_model.load_weights('Brain(MLAF-CapsNet)_weights.best.hdf5')

In [ ]:
## Print Primary Capsule Summary

In [ ]:
pc_model.summary()

In [ ]:
## Print DigitCapss Summary

In [ ]:
digitcaps_model.summary() #Inspect the digitCaps model without decoder

In [ ]:
## Print Decoder Summary

In [ ]:
decoder.summary() #Inspect the decoder model without decoder

In [ ]:
## Import Model

In [ ]:
import os 
file_paths = ["Brain(MLAF-CapsNet)_model.h5"]
for file_path in file_paths:
    w = os.stat(file_path).st_size
    print("{:30} {:4.2f} MB".format(file_path,w/(1024.0**2)))

In [ ]:
## Train and print Confusion Matrix

In [ ]:

from sklearn.metrics import classification_report, confusion_matrix,precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc
import pandas as pd

import seaborn as sn
from sklearn.preprocessing import label_binarize
from scipy import interp
from scipy.interpolate import interp1d
from itertools import cycle


## Evaluating the Model
y_pred = eval_model.predict(x_test)

# Print prediction
#print('pred_y',y_pred[:15])
#NUM_CLASSES=[0,1,2,3,4]

#print("y_test", y_test[:15])


y_pred = np.array(y_pred[0])
#y_pred=np.array(y_pred)



round_pred = y_pred.argmax(axis=1)


y_test=np.argmax(y_test, axis=1)

cnf_matrix = confusion_matrix(y_test, round_pred)
#print(cnf_matrix)

#kk = pd.DataFrame(cnf_matrix, range(3), range(3))
kk = pd.DataFrame(cnf_matrix, range(NUM_CLASSES), range(NUM_CLASSES))
sn.heatmap(kk, annot=True, annot_kws={'size':12}, fmt='g', cmap='Blues')
plt.savefig('eye CM plot.jpg', dpi=1200)
plt.show()

#cnf_matrix = confusion_matrix(y_test, round_pred)
    #print(cnf_matrix)


In [ ]:
## ROC curves

In [ ]:

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
'''



    #fpr, tpr, _ = roc_curve(y_test, np.argmax(round_pred, axis=0))
    fpr, tpr, _ = roc_curve(y_test, round_pred)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(11,8))
    lw = 10
    plt.plot(fpr, tpr, color='darkorange',lw=lw, label='ROC curve (auc = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.savefig('roc2.png')
'''
y_test=label_binarize(y_test, classes=[i for i in range(NUM_CLASSES)])
# First aggregate all false positive rates
print("y_test1[:, 1]",y_test[:, 1])

print("y_pred.argmax(axis=1)",y_pred.argmax(axis=1))
#hj=y_pred.argmax(axis=1)
#hj=label_binarize(round_pred , classes=[i for i in range(NUM_CLASSES)])
#print("y_pred_linarize",hj)

for i in range(NUM_CLASSES):
        fpr[i], tpr[i],threshold = roc_curve(y_test[:, i],  y_pred[:, i])

        #print("fpr[i]",fpr[i])
        #print("tpr[i]",tpr[i])
        #print("threshold", threshold)
        roc_auc[i] = auc(fpr[i], tpr[i])

    #print("y_test.shape", y_test.shape)
    #print("hj.shape",hj.shape)
    #print("hj1.shape",hj1.shape)
    #print("y_test[:,1].shape", y_test[:,1].shape)

    #Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test[:,i].ravel(), y_pred[:, i].ravel())

roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

all_fpr = np.unique(np.concatenate([fpr[i] for i in range(NUM_CLASSES)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(NUM_CLASSES):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
mean_tpr /=NUM_CLASSES

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
lw=1.5
# Plot all ROC curves
plt.figure()
plt.figure(figsize=(9, 10))
'''
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=4)
'''
colors = cycle(['aqua', 'darkorange', 'cornflowerblue','purple', 'navy', 'teal','turquoise', 'yellow', 'green','red'])

for i, color in zip(range(NUM_CLASSES), colors):
        #f=interp1d(fpr[i], tpr[i], kind='cubic')
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC curve of class {0} (area = {1:0.2f})'
                 ''.format(i, roc_auc[i]))


plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right",prop=dict(size=20), fontsize=20)
plt.tight_layout()
plt.savefig('eye-ROC plot.jpg', dpi=1200)
plt.show()




   

In [ ]:
## Precision Recall Curves

In [ ]:

##################################PRECISION RECALL CURVE###############################################################
    # For each class
precision = dict()
recall = dict()
average_precision = dict()
for i in range(NUM_CLASSES):
        precision[i], recall[i], _ = precision_recall_curve(y_test[:, i],  y_pred[:,i])

        average_precision[i] = average_precision_score(y_test[:, i], y_pred[:,i])

    # A "micro-average": quantifying score on all classes jointly
precision["micro"], recall["micro"], _ =precision_recall_curve(y_test[:,1].ravel(),y_pred[:, 1].ravel())

average_precision["micro"] = average_precision_score(y_test[:,i],y_pred[:,i], average="micro")

print('Average precision score, micro-averaged over all classes: {0:0.2f}'
          .format(average_precision["micro"]))

    # setup plot details
colors = cycle(['navy', 'turquoise', 'darkorange', 'cornflowerblue', 'teal','red', 'green', 'purple', 'brown', 'cyan'])

plt.figure(figsize=(9, 10))
f_scores = np.linspace(0.2, 0.8, num=4)
lines = []
labels = []
for f_score in f_scores:
        x = np.linspace(0.01, 1)
        y = f_score * x / (2 * x - f_score)
        l, = plt.plot(x[y >= 0], y[y >= 0], color='gray', alpha=0.2)
        plt.annotate('f1={0:0.1f}'.format(f_score), xy=(0.9, y[45] + 0.02))

lines.append(l)
labels.append('iso-f1 curves')
l, = plt.plot(recall["micro"], precision["micro"], color='gold', lw=2)
lines.append(l)
    #labels.append('micro-average Precision-recall (area = {0:0.2f})'
    #              ''.format(average_precision["micro"]))

for i, color in zip(range(NUM_CLASSES), colors):
        l, = plt.plot(recall[i], precision[i], color=color, lw=2)
        lines.append(l)
        labels.append('Precision-recall for class {0} (area = {1:0.2f})'
                      ''.format(i, average_precision[i]))

fig = plt.gcf()
fig.subplots_adjust(bottom=0.25)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Extension of Precision-Recall curve to multi-class')
plt.legend(lines, labels, loc="lower left", prop=dict(size=10), fontsize=20)
    #(0, -.38)
plt.savefig('eye 10-PR plot.jpg', dpi=1200)
plt.show()
